Phase 1 Requirements  
Preprocessing of the captured image. (10 marks) 
Credit card frame detection. (5 marks) 
Straightening and cropping the picture such that it only contains the undistorted PAN. (5 marks) 

Imports

In [111]:
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

 define a dictionary that maps the first digit of a credit card
number to the credit card type

In [112]:
FIRST_NUMBER = {
	"3": "American Express",
	"4": "Visa",
	"5": "MasterCard",
	"6": "Discover Card"
}

Getting the Refrence image 
|||
||
|

Getting the test image
(Getting the image + Grayscale it )

In [113]:
image = cv2.imread('./testcases/01 - Straightforward.jpg')
#image = cv2.imread('./testcases/02 - You can do it.jpg')
#image = cv2.imread('./testcases/03 - Should be okay.jpg')
#image = cv2.imread('./testcases/04 - Still ok, I hope.jpg')
#image = cv2.imread('./testcases/05 - Looks cool, hope it runs cool too.jpg') 
#image = cv2.imread('./testcases/06 - Hatetlewe7 hatlewe7.jpg')
#image = cv2.imread('./testcases/07 - Hatet3eweg hat3eweg.jpg')
#image = cv2.imread('./testcases/08 - Ew3a soba3ak ya3am.jpg')
#image = cv2.imread('./testcases/09 - El spero spathis we23et 3aaaa.jpg')
#image = cv2.imread('./testcases/10 - Mal7 w Felfel.jpg') 
# image = cv2.imread('./testcases/11 - Ya setty ew3i.jpg')
# image = cv2.imread('./testcases/12 - weewooweewoo.jpg') 
# image = cv2.imread('./testcases/13 - Matozbot el camera ya Kimo.jpg')
# image = cv2.imread('./testcases/14 - 2el noor 2ata3.jpg')
#image = cv2.imread('./testcases/15 - Compresso Espresso.jpg')
# image = cv2.imread('./testcases/16 - Sheel el kart yastaaaa.jpg')


image = imutils.resize(image, width=300)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Test images 
cv2.imshow("Original", image)
cv2.imshow("Gray", gray)

# Prevent this window from closing
cv2.waitKey(0)
cv2.destroyAllWindows()

Adding Kernals 

In [114]:
# initialize a rectangular (wider than it is tall) and square
# structuring kernel
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

Applying Tophat morph

In [115]:
# apply a tophat (whitehat) morphological operator to find light
# regions against a dark background (i.e., the credit card numbers)
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel)
cv2.imshow("Tophat", tophat)

# Prevent this window from closing
cv2.waitKey(0)
cv2.destroyAllWindows()

Compute the Gradient along the x-direction(Vertical Edges )

In [116]:
# compute the Scharr gradient of the tophat image, then scale
# the rest back into the range [0, 255]
gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,
	ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")

cv2.imshow("Sobel X", gradX)
cv2.waitKey(0)
cv2.destroyAllWindows()

Credit Card digit finding algorithm

In [117]:
# apply a closing operation using the rectangular kernel to help
# cloes gaps in between credit card number digits, then apply
# Otsu's thresholding method to binarize the image
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
thresh = cv2.threshold(gradX, 0, 255,
	cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
# apply a second closing operation to the binary image, again
# to help close gaps between credit card number regions
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)

cv2.imshow("Sobel X", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()